In [ ]:
import numpy as np, math, matplotlib.pyplot as plt

def nkg(r, r_m, s, N_e=1e5):
    """NKG lateral density ρ_e(r)."""
    C = N_e/(2*np.pi*r_m**2) * math.gamma(4.5-s)/(math.gamma(s)*math.gamma(4.5-2*s))
    return C * (r/r_m)**(s-2) * (1 + r/r_m)**(s-4.5)

r   = np.logspace(0.1, 2, 400)   # 1–1000 m
rho = nkg(r, r_m=10, s=0.1)

plt.loglog(r, rho)
plt.xlabel("r [m]");  plt.ylabel(r"$\rho_e(r)$")
plt.grid(True, which="both", ls="--", lw=0.5)
plt.tight_layout();  plt.show()

In [ ]:
import ana
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ROOT
import os

ROOT.gSystem.Load("/Users/macbook/PycharmProjects/pythonProject/searchMonopole/paper/LHAASO_Monopole/Filt_Event/bin/libLHEvent.so")   # 打开 ROOT 文件
ED_pos_file = "/Users/macbook/PycharmProjects/pythonProject/searchMonopole/paper/LHAASO_Monopole/Filt_Event/config/ED_all.txt"
MD_pos_file = "/Users/macbook/PycharmProjects/pythonProject/searchMonopole/paper/LHAASO_Monopole/Filt_Event/config/MD_all.txt"
ED_pos = pd.read_csv(ED_pos_file, sep=" ", skiprows=1)
MD_pos = pd.read_csv(MD_pos_file, sep=" ", skiprows=1)

file_DIR=f"./Dataset_Filted/Simulation/gamma/1e3_1e4"
file=f"combined_gamma_1e3_1e4.root"
infile=os.path.join(file_DIR,file)

file_path=infile
f = ROOT.TFile.Open(file_path)
event = ROOT.LHFiltedEvent()
tree = f.Get("filted_tree") 
tree.SetBranchAddress("FiltedEvent", event)
nentries=tree.GetEntries()
n=100
tree.GetEntry(n)
sim_event=ana.read_filted_event(event,num=n,nentries=nentries)
hitE=sim_event.get_hitE()
hitM=sim_event.get_hitM()
core=sim_event.get_core()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4.6))
scatter1 = ax1.scatter(hitE['x'], hitE['y'], c=hitE['pe'], s=10)
ax1.scatter(core[0], core[1], c='r', s=100, marker='x')
ax1.set_title('E Hits')
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
fig.colorbar(scatter1, ax=ax1, label='PE (Photoelectrons)')

scatter2 = ax2.scatter(hitM['x'], hitM['y'], c=hitM['pe'], s=10)
ax2.scatter(core[0], core[1], c='r', s=100, marker='x')
ax2.set_title('M Hits')
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
fig.colorbar(scatter2, ax=ax2, label='PE (Photoelectrons)')

plt.tight_layout()
plt.show()
plt.close()



In [ ]:
import numpy as np
from torch_geometric.data import Data


def hits_to_image(hits, grid_size=64):
    """将 (n_hits, 3) -> (grid_size, grid_size) 图像"""
    img = np.zeros((grid_size, grid_size), dtype=np.float32)
    
    if hits.size == 0:
        return img

    # 归一化到 [0, grid_size-1]
    x = hits[:, 0]
    y = hits[:, 1]
    pe = hits[:, 2]

    x_idx = np.clip(((x - x.min()) / (x.max() - x.min() + 1e-8) * (grid_size - 1)).astype(int), 0, grid_size - 1)
    y_idx = np.clip(((y - y.min()) / (y.max() - y.min() + 1e-8) * (grid_size - 1)).astype(int), 0, grid_size - 1)

    for xi, yi, val in zip(x_idx, y_idx, pe):
        img[yi, xi] += val

    return img

# 加载数据
data = np.load("./Dataset_Filted/Simulation/gamma/1e3_1e4/train_dataset_gamma_1e3_1e4_run000.npz", allow_pickle=True)
hitsE = data["hitsE"]
hitsM = data["hitsM"]
labels = data["labels"]
print(hitsE[0])
# 转成 CNN 输入 [N, 2, H, W]  (2 通道：HitsE 和 HitsM)
grid_size = 64
X = np.array([
    np.stack([
        hits_to_image(hE, grid_size),
        hits_to_image(hM, grid_size)
    ], axis=0)
    for hE, hM in zip(hitsE, hitsM)
], dtype=np.float32)

y = np.array(labels, dtype=np.int64)

print("CNN 输入形状:", X.shape)  # (事件数, 2, 64, 64)
print("标签形状:", y.shape)



In [ ]:
import torch
from torch_geometric.data import Data
graph_list = []
for hE, hM, label in zip(hitsE, hitsM, labels):
    # 节点特征 [n_nodes, 3]
    nodes = np.vstack([hE, hM])
    x_tensor = torch.tensor(nodes, dtype=torch.float32)

    # 暂时用 fully-connected edges（实际可以用邻近构图）
    n_nodes = nodes.shape[0]
    edge_index = torch.combinations(torch.arange(n_nodes), r=2).T
    edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)

    data = Data(x=x_tensor,
                edge_index=edge_index,
                y=torch.tensor([label], dtype=torch.long))
    graph_list.append(data)

In [ ]:
data = np.load("./Dataset_Filted/Simulation/gamma/1e3_1e4/train_dataset_gamma_1e3_1e4_run000.npz", allow_pickle=True)
hitsE = data["hitsE"]
hitsM = data["hitsM"]
labels = data["labels"]

sample_size = 20000
random_indices = np.random.choice(len(labels), size=sample_size, replace=False)
sampled_hitsE = hitsE[random_indices]
sampled_hitsM = hitsM[random_indices]
sampled_labels = labels[random_indices]